In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re

In [2]:
url = 'https://sofifa.com/players?offset=0'

In [3]:
def soup_maker(url):
    r = requests.get(url)
    markup = r.content
    soup = bs(markup, 'lxml')
    return soup

# Scratch Work

In [4]:
soup = soup_maker(url)

In [5]:
scraped_data = []
tbl = soup.find('table',{'class':'table table-hover persist-area'})
tbdy = tbl.find('tbody')
all_a = tbdy.find_all('a', {'rel':None})
player_details = {}
# players on every other index
for i, lnk in enumerate(all_a):
    if i % 2 == 0:
        player_details['short_name'] = lnk.text
        #player_details.update(get_player_details('http://sofifa.com' + lnk['href']))
        player_details['link'] = 'http://sofifa.com' + lnk['href']
        print(player_details)
        scraped_data.append(player_details)

{'short_name': 'L. Messi', 'link': 'http://sofifa.com/player/158023/lionel-messi/'}
{'short_name': 'Cristiano Ronaldo', 'link': 'http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/'}
{'short_name': 'Neymar Jr', 'link': 'http://sofifa.com/player/190871/neymar-da-silva-santos-jr/'}
{'short_name': 'De Gea', 'link': 'http://sofifa.com/player/193080/david-de-gea-quintana/'}
{'short_name': 'K. De Bruyne', 'link': 'http://sofifa.com/player/192985/kevin-de-bruyne/'}
{'short_name': 'E. Hazard', 'link': 'http://sofifa.com/player/183277/eden-hazard/'}
{'short_name': 'L. Modrić', 'link': 'http://sofifa.com/player/177003/luka-modric/'}
{'short_name': 'L. Suárez', 'link': 'http://sofifa.com/player/176580/luis-suarez/'}
{'short_name': 'H. Kane', 'link': 'http://sofifa.com/player/202126/harry-kane/'}
{'short_name': 'J. Oblak', 'link': 'http://sofifa.com/player/200389/jan-oblak/'}
{'short_name': 'R. Lewandowski', 'link': 'http://sofifa.com/player/188545/robert-lewandowski/'}
{'short_name': 'T. 

In [6]:
all_deets = {}
soup = soup_maker('http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/')
plyr_info = soup.find('div', {'class':'player'})

# find_plyr_info
plyr_dat = {} # prepare dictionary for player data
ply_inf = plyr_info.find('div', {'class':'meta'}) # grab section with player data
infos = ply_inf.text
data_str = infos[infos.index('Age') + 4:] # grab all data in str to right of Age like height and weight

# set variables
plyr_dat['pref_pos'] = ply_inf.find('span').text
plyr_dat['full_name'] = ply_inf.text[0:ply_inf.text.find(plyr_dat['pref_pos'])-2].strip()
plyr_dat['age'] = int(data_str[:2])
plyr_dat['height'] = data_str[data_str.index(')') + 2:].split(' ')[0].replace('\"','')
plyr_dat['weight'] = data_str[data_str.index(')') + 2:].split(' ')[1]
print(plyr_dat)

{'pref_pos': 'ST', 'full_name': 'C. Ronaldo dos Santos Aveiro (Cristiano Ronaldo dos Santos Aveiro)', 'age': 33, 'height': "6'2", 'weight': '183lbs'}


In [7]:
plyr_data = {}
soup = soup_maker('http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/')
plyr_stats = soup.find('div', {'class':'stats'})
plyr_val = plyr_stats.text[plyr_stats.text.find('€'):].split('\n')[0]
info = re.findall('\d+', plyr_stats.text)
plyr_data['rating'] = int(info[0])
plyr_data['potential'] = int(info[1])

if 'M' in plyr_val:
    plyr_data['value'] = int(plyr_val[1:plyr_val.index('M')])*1000000
elif 'K' in plyr_val:
    plyr_data['value'] = int(plyr_val[1:plyr_val.index('K')])*1000

plyr_data['wage'] = int(info[3])*1000

print(plyr_data)

{'rating': 94, 'potential': 94, 'value': 77000000, 'wage': 405000}


# Ok, now let's turn this work into functions

In [8]:
def gather_basic_info(soup):
    plyr_dat = {} # prepare dictionary for player data
    ply_inf = soup.find('div', {'class':'meta'}) # grab section with player data
    infos = ply_inf.text
    data_str = infos[infos.index('Age') + 4:] # grab all data in str to right of Age like height and weight

    # set variables
    plyr_dat['pref_pos'] = ply_inf.find('span').text
    plyr_dat['full_name'] = ply_inf.text[0:ply_inf.text.find(plyr_dat['pref_pos'])-2].strip()
    plyr_dat['age'] = int(data_str[:2])
    plyr_dat['height'] = data_str[data_str.index(')') + 2:].split(' ')[0].replace('\"','')
    plyr_dat['weight'] = data_str[data_str.index(')') + 2:].split(' ')[1]
    return(plyr_dat)

In [9]:
def gather_player_stats(soup):
    # parse stats section and determine rating, potential, value, and wage
    plyr_data = {}
    plyr_val = soup.text[soup.text.find('€'):].split('\n')[0]
    info = re.findall('\d+', soup.text)
    plyr_data['rating'] = int(info[0])
    plyr_data['potential'] = int(info[1])

    # check units of their value
    if 'M' in plyr_val:
        plyr_data['value'] = int(float(plyr_val[1:plyr_val.index('M')])*1000000)
    elif 'K' in plyr_val:
        plyr_data['value'] = int(float(plyr_val[1:plyr_val.index('K')])*1000)

    # wage is always given in thousands
    plyr_data['wage'] = int(info[3])*1000
    return(plyr_data)

In [10]:
def get_player_details(soup):
    # take in player url and build dictionary of player info
    all_deets = {}
    soup = soup_maker(soup)
    
    # gather basic name, height, weight stats
    plyr_info = soup.find('div', {'class': 'player'})
    all_deets.update(gather_basic_info(plyr_info))
    
    # gather rating, value, wage, etc
    plyr_stats = soup.find('div', {'class': 'stats'})
    all_deets.update(gather_player_stats(plyr_stats))
    
    return(all_deets)

# Now let's create our dataframe

In [11]:
def scrape_sofifa():
    # completes entire scraping process, can take minutes to run
    scraped_data = []
    page_indexes = [i*60 for i in range(5)]
    
    # iterate across 100 different player pages
    for page_index in page_indexes:
        url = 'https://sofifa.com/players?offset=' + str(page_index)
        soup = soup_maker(url)
        
        # gather all player details for this page
        tbl = soup.find('table',{'class':'table table-hover persist-area'})
        tbdy = tbl.find('tbody')
        all_a = tbdy.find_all('a', {'rel':None})
        player_details = {}
        
        # players are on every other index
        for i, lnk in enumerate(all_a):
            if i % 2 == 0:
                player_details['short_name'] = lnk.text
                player_details['link'] = 'http://sofifa.com' + lnk['href']
                player_details.update(get_player_details('http://sofifa.com' + lnk['href']))
                print(player_details)
                scraped_data.append(player_details)
    
    return(scraped_data)

In [ ]:
import pandas as pd
df = pd.DataFrame(scrape_sofifa())
df.to_csv('scraped_sofifa.csv', index=False)

{'short_name': 'L. Messi', 'link': 'http://sofifa.com/player/158023/lionel-messi/', 'pref_pos': 'CF', 'full_name': 'Lionel Messi (Lionel Andrés Messi Cuccittini)', 'age': 31, 'height': "5'7", 'weight': '159lbs', 'rating': 94, 'potential': 94, 'value': 110500000, 'wage': 5000}
{'short_name': 'Cristiano Ronaldo', 'link': 'http://sofifa.com/player/20801/c-ronaldo-dos-santos-aveiro/', 'pref_pos': 'ST', 'full_name': 'C. Ronaldo dos Santos Aveiro (Cristiano Ronaldo dos Santos Aveiro)', 'age': 33, 'height': "6'2", 'weight': '183lbs', 'rating': 94, 'potential': 94, 'value': 77000000, 'wage': 405000}
{'short_name': 'Neymar Jr', 'link': 'http://sofifa.com/player/190871/neymar-da-silva-santos-jr/', 'pref_pos': 'LW', 'full_name': 'Neymar da Silva Santos Jr. (Neymar da Silva Santos Junior)', 'age': 26, 'height': "5'9", 'weight': '150lbs', 'rating': 92, 'potential': 92, 'value': 108000000, 'wage': 290000}
{'short_name': 'De Gea', 'link': 'http://sofifa.com/player/193080/david-de-gea-quintana/', 'pre

{'short_name': 'P. Aubameyang', 'link': 'http://sofifa.com/player/188567/pierre-emerick-aubameyang/', 'pref_pos': 'ST', 'full_name': 'Pierre-Emerick Aubameyang', 'age': 29, 'height': "6'2", 'weight': '176lbs', 'rating': 88, 'potential': 88, 'value': 59000000, 'wage': 265000}
{'short_name': 'M. Reus', 'link': 'http://sofifa.com/player/188350/marco-reus/', 'pref_pos': 'CAM', 'full_name': 'Marco Reus', 'age': 29, 'height': "5'11", 'weight': '157lbs', 'rating': 88, 'potential': 88, 'value': 58000000, 'wage': 130000}
{'short_name': 'M. Hummels', 'link': 'http://sofifa.com/player/178603/mats-hummels/', 'pref_pos': 'CB', 'full_name': 'Mats Hummels (Mats  Hummels)', 'age': 29, 'height': "6'3", 'weight': '203lbs', 'rating': 88, 'potential': 88, 'value': 46000000, 'wage': 160000}
{'short_name': 'G. Bale', 'link': 'http://sofifa.com/player/173731/gareth-bale/', 'pref_pos': 'RW', 'full_name': 'Gareth Bale (Gareth Frank Bale)', 'age': 28, 'height': "6'1", 'weight': '181lbs', 'rating': 88, 'potentia